In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.utils import get_file

from deepcell.utils.plot_utils import create_rgb_image
from deepcell.datasets import Dataset
from deepcell_spots.applications import SpotDecoding, SpotDetection
from deepcell_spots.preprocessing_utils import min_max_normalize
from deepcell_spots.postprocessing_utils import max_cp_array_to_point_list_max
from deepcell_spots.multiplex import extract_spots_prob_from_coords_maxpool

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [2]:
def load_data(self, path=None):
    path = path if path else self.path
    
    basepath = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
    prefix = path.split(os.path.sep)[:-1]
    data_dir = os.path.join(basepath, *prefix) if prefix else basepath
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    elif not os.path.isdir(data_dir):
        raise IOError('{} exists but is not a directory'.format(data_dir))

    path = get_file(path,
                    origin=self.url,
                    file_hash=self.file_hash)
    
    data = np.load(path)
    spots_image = data['spots_image']
    spots_image = np.swapaxes(spots_image, 3, 0)
    
    return spots_image

def load_codebook(self, path=None):
    path = path if path else self.path
    
    basepath = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
    prefix = path.split(os.path.sep)[:-1]
    data_dir = os.path.join(basepath, *prefix) if prefix else basepath
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    elif not os.path.isdir(data_dir):
        raise IOError('{} exists but is not a directory'.format(data_dir))

    path = get_file(path,
                    origin=self.url,
                    file_hash=self.file_hash)
    df = pd.read_csv(path, index_col=0)

    return df

Dataset.load_data = load_data
Dataset.load_codebook = load_codebook

In [3]:
datafile = Dataset(
    path='MERFISH_example.npz',
    url='https://deepcell-data.s3.us-west-1.amazonaws.com/spot_detection/multiplex/Moffitt/MERFISH_example.npz',
    file_hash='2cd7ce177b503fd0873125784097622b',
    metadata={})
spots_image = datafile.load_data()
spots_image.shape

(20, 500, 500, 1)

## Spot Detection

This section can be substituted for any other spot detection method. For simplicity, we will use the `SpotDetection` application to perform spot detection.

In [4]:
app = SpotDetection()
app.postprocessing_fn = None

2023-03-18 16:15:38.023880: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 16:15:38.627506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37502 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


In [5]:
pred = app.predict(spots_image, clip=True, threshold=0.5)

2023-03-18 16:15:44.110670: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2023-03-18 16:15:45.631286: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [6]:
output_image = pred['classification'][:,...,1:2]
output_image = np.swapaxes(output_image, 0, 3)
output_image.shape

(1, 500, 500, 20)

In [7]:
spots_threshold = 0.9
max_proj_images = np.max(output_image[...,40:-40,40:-40,:20], axis=-1)
spots_locations = max_cp_array_to_point_list_max(max_proj_images,
                                                 threshold=spots_threshold, min_distance=1)

In [8]:
maxpool_extra_pixel_num=0
spots_intensities = extract_spots_prob_from_coords_maxpool(
    output_image[...,40:-40,40:-40,:20], spots_locations, extra_pixel_num=maxpool_extra_pixel_num)
spots_intensities_vec = np.concatenate(spots_intensities)
spots_locations_vec = np.concatenate([np.concatenate(
    [item, [[idx_batch]] * len(item)], axis=1)
    for idx_batch, item in enumerate(spots_locations)])

spots_intensities_vec.shape, spots_locations_vec.shape

((1205, 20), (1205, 3))

## Spot Decoding

In [9]:
codebook = Dataset(
    path='codebook.csv',
    url='https://deepcell-data.s3.us-west-1.amazonaws.com/spot_detection/multiplex/Moffitt/codebook.csv',
    file_hash='70a9a4690c3e516ed6f4b0cfe4911a60',
    metadata={})

df_barcodes = codebook.load_codebook()
rounds = 10
channels = 2

In [10]:
dec_app = SpotDecoding(df_barcodes=df_barcodes,
                       rounds=rounds,
                       channels=channels)

The `SpotDecoding` application takes a vector of spot intensities with the shape (num. spots, `rounds`*`channels`) as its input. 

In [11]:
decoding_result = dec_app.predict(spots_intensities_vec)

`spots_locations_vec` can be used to populate a DataFrame, which resembles the Polaris output.

In [12]:
df = pd.DataFrame()
df[['x', 'y', 'batch_id']] = spots_locations_vec.astype(np.int32)
df['cell_id'] = None
for name, val in decoding_result.items():
    df[name] = val
    
df

,x,y,batch_id,cell_id,probability,predicted_id,predicted_name
0,259,242,0,None,1.000000,5,Txndc5
1,162,251,0,None,0.999451,54,Stmn1
2,193,175,0,None,0.999936,4,Cps1
3,153,50,0,None,1.000000,4,Cps1
4,77,262,0,None,1.000000,4,Cps1
...,...,...,...,...,...,...,...
1200,270,354,0,None,0.831650,5,Txndc5
1201,137,318,0,None,0.927471,266,Background
1202,82,174,0,None,0.981118,266,Background
1203,233,385,0,None,0.949393,266,Background
